## matplotlib (plotting) and SciPy (scientific algorithms) libraries 

The SciPy library provides an emormous variety of scientific algorithms, exposed in the Python language,
but often written in low-level languages like C and Fortran to empower us with the fastest performance. 
Many of these algorithms are too "heavy" to be implemented in the basic NumPy library; however, SciPy
is built upon the data structures and operations enabled by NumPy and the two libraries are often
used side-by-side.

Let's start with a common problem in many scientific disciplines -- calculating the Voronoi diagram
for a set of data points. Since this problem falls into the category of "computational geometry,"
the SciPy developers have placed it in the scipy.spatial submodule with other spatial algorithms
and data structures.

In [ ]:
import numpy as np
import scipy
import scipy.spatial
import matplotlib
matplotlib.use('agg')
%matplotlib inline
import matplotlib.pyplot as plt

# we start off with a set of random points, which may for example represent
# standing trees in a forest after a forest fire event
tree_positions = np.random.randn(15, 2)

In [ ]:
# at this point, it is already helpful to first take a look at the positions of the trees on the 2D "map"
# let's start using matplotlib for this work

# first generate a figure object
fig = plt.figure()

# then create an axis object (which can be used for adding the data and labels)
ax = fig.add_subplot(111) # 111 means 1 row, 1 column, 1st plot in our figure "panel"

# scatter plot the 2D tree position data
ax.scatter(tree_positions[...,0], # x coordinates
           tree_positions[...,1]) # y coordinates

ax.set_title("Trees Remaining After Forest Fire")
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)

In [ ]:
# The Voronoi diagram will tell us which parts of the forest are closest to which tree
# This can be used i.e., as an estimate of the amount of area occupied by each tree (the area around it)

vor = scipy.spatial.Voronoi(tree_positions)

# it is such a common operation to plot the Voronoi diagram of a set of 2D generators
# that SciPy can plot the diagram (using matplotlib under the hood) directly
# from the vor object above

fig_vor = scipy.spatial.voronoi_plot_2d(vor)
axis = fig_vor.get_axes()[0]
axis.set_xlim(-3, 3)
axis.set_ylim(-3, 3)

In [ ]:
# let's take a step back though & see if we can generate a similar plot using the
# vor object and matplotlib (instead of using voronoi_plot_2d directly), as an
# exercise in learning about matplotlib and SciPy

# the blue points above are the "generators," while the orange points are the Voronoi
# vertices bounding the Voronoi regions

fig_manual = plt.figure()
ax_manual = fig_manual.add_subplot(111)

# there's a convenient way to access the Voronoi vertices in SciPy
vor_vertices = vor.vertices
ax_manual.scatter(vor_vertices[...,0], # x coords
           vor_vertices[...,1], # y coords
           color='orange')

# to connect the Voronoi vertices into the Voronoi edges (the polygon
# edges that enclose Voronoi regions) we can use the "ridges:""
vor_ridges = vor.ridge_vertices

# the above ridges are actually indices of Voronoi vertices, so we
# will iterate through and plot accordingly
for edge in vor_ridges:
    if -1 in edge:
        # some edges can continue to infinity
        # those are dashed lines in voronoi_plot_2d, but let's
        # ignore them here
        continue
    edge_start = vor_vertices[edge[0]]
    edge_end = vor_vertices[edge[1]]
    ax_manual.plot([edge_start[0], edge_end[0]], # the two x coords
                   [edge_start[1], edge_end[1]], # the two y coords
                   color='black')
    
ax_manual.set_xlim(-3, 3)
ax_manual.set_ylim(-3, 3)

# let's add the generators back in as well, to facilitate comparison
# with plot above
ax_manual.scatter(tree_positions[...,0], # x coordinates
                  tree_positions[...,1], # y coordinates
                  color='blue') 

So, the plots look pretty similar whether we use matplotlib manually in conjunction with SciPy or if we use the the built-in convenience function voronoi_plot_2d()

In [ ]:
# if we instead wanted to calculate the area of the entire forest we could do that quite easily
# with SciPy as well by wrapping all the trees with an "elastic band" (the Convex Hull)

hull = scipy.spatial.ConvexHull(tree_positions)
forest_area = hull.area
forest_area

In [ ]:
# to confirm the elastic band nature of the Convex Hull, let's plot it using
# matplotlib as usual
hull_fig = plt.figure()
hull_ax = hull_fig.add_subplot(111)
for simplex in hull.simplices:
    hull_ax.plot(hull.points[simplex, 0],
                 hull.points[simplex, 1],
                 '-',
                 lw=6)
    
# and restore scatter of the tree positions as well
hull_ax.scatter(tree_positions[...,0], # x coordinates
                tree_positions[...,1], # y coordinates
                color='black',
                s=200) 

Now, perhaps we've discovered that the region affected by the forest fire can actually be estimated as the area between
a curve defined by a function and a roughly straight line ocean / coastal boundary.

Let's say this may be expressed as the following definite integral:

$$\int_{-3}^{3} (x^2 + 5x + 30) \,dx$$

In [ ]:
# if we want to estimate the numerical result of that definite integral (area affected by forest fire)
# we'll want to use scipy.integrate.quad()
import scipy.integrate

# start by defining the function of interest
def forest_func(x):
    return x ** 2 + (5 * x) + 30

# call quad() using the function name and the limits of the definite integral
area_estimate, upper_bound_error = scipy.integrate.quad(forest_func, -3, 3)
area_estimate, upper_bound_error

In [ ]:
# let's plot the function over the limits of integration and shade in the
# area we just estimated

from matplotlib.patches import Polygon
fig_integrate = plt.figure()
ax_integrate = fig_integrate.add_subplot(111)

# plot the function over a slightly wider limit range for clarity
x = np.linspace(-5, 5)
y = forest_func(x)
ax_integrate.plot(x, y, 'black')

# for the shaded region
# see: https://matplotlib.org/examples/showcase/integral_demo.html
ix = np.linspace(-3, 3)
iy = forest_func(ix)
verts = [(-3, 0)] + list(zip(ix, iy)) + [(3, 0)]
poly = Polygon(verts, facecolor='blue', edgecolor='blue', alpha=0.3)
ax_integrate.add_patch(poly)

If we want to find the point on the curve (forest region) that is closest to the ocean / coastal boundary we might
want to find the minimum of the function we just integrated. There are various ways to do this, but for demonstration
purposes let's try to *minimize* our function using SciPy. Specifically, we'll use `scipy.optimize.minimize`

In [ ]:
import scipy.optimize

# this is a pretty naive optimization (I rarely use scipy.optimize)
# we haven't specified the algorithm to use and so on
# but maybe that's a good thing for clarity anyway
optimization_result = scipy.optimize.minimize(fun=forest_func,
                                              x0=250) # perhaps we're really bad at guessing the solution!

optimization_result                                          

`x` is the solution of the minimization / optimization, and `x = -2.5` looks about right for the minimum of our function above

likewise, `fun` is the `y` value of our "objective function" at the minimum `x` value; again, the value of `23.75` looks about right based on visual inspection of this quadratic

Now, let's say we want to take a close look at how our forest boundary is defined based on the set of discrete data points we have access to.
Perhaps we've discovered that we actually only have about 10 data points, which isn't very many

In [ ]:
x = np.linspace(-5, 5, num=10)
y = forest_func(x)

# let's try two types of interpolation to connect our boundary points
import scipy.interpolate

# we first generate interpolation functions, which can then
# operate on i.e., a denser range of x values
f_linear = scipy.interpolate.interp1d(x, y)
f_cubic = scipy.interpolate.interp1d(x, y, kind='cubic')

x_new = np.linspace(-5, 5, num=15) # denser range for the interpolation plots

# plot the interpolations
# see: https://docs.scipy.org/doc/scipy/reference/tutorial/interpolate.html

fig_interp = plt.figure()
ax_interp = fig_interp.add_subplot(111)

ax_interp.plot(x, y, 'o',
               x_new, f_linear(x_new), '-',
               x_new, f_cubic(x_new), '--')

ax_interp.legend(['data', 'linear', 'cubic'], loc='best', fontsize=20)

# focus on minimum of the quadratic to emphasize interpolation differences
ax_interp.set_ylim(20, 32)

fig_interp.set_size_inches(8, 8)


So, perhaps cubic interpolation does a slightly better job of approximating the function in this case.

Let's imagine we've now been directed to study the behavioral / environmental impact of the forest fire in the affected area.

Since the forest in question normally has two primary animals that generate noise at regular intervals during nighttime hours,
we've gone ahead and recorded some audio data from several evenings. We'd now like to convert this periodic / time-domain
data to frequency-based data so that we can confirm the species of animals generating the noises.

In [ ]:
# for this particular application (converting periodic time domain data to frequency domain data, Fourier transforms are appropriate)
import scipy.fftpack

# example inspired by: https://docs.scipy.org/doc/scipy/reference/tutorial/fftpack.html
n_samples = 600
sample_spacing = 1.0 / 800.0
time_values = np.linspace(0.0, n_samples * sample_spacing, n_samples)
recorded_values = np.sin(50.0 * 2.0*np.pi*time_values) + 0.5*np.sin(80.0 * 2.0*np.pi*time_values)

# convert to frequency domain data using a Fast Fourier Transform
y_freq = scipy.fftpack.fft(recorded_values)
x_freq = np.linspace(0.0, 1.0/(2.0 * sample_spacing), n_samples // 2)

fig_fft = plt.figure()
ax_fft = fig_fft.add_subplot(111)

# typically only the positive fft values are plotted

ax_fft.plot(x_freq,
            2.0 / n_samples * np.abs(y_freq[0:n_samples//2]))
ax_fft.grid()
ax_fft.set_xlabel('Frequency', fontsize=20)
ax_fft.set_ylabel('Intensity', fontsize=20)

Perhaps we could identify the two animal species that remain after the forest fire based on the audio frequency data above.

It turns out that a "citizen scientist" has managed to capture a photo of one of these suspected animal species, and while
we think we know what it is, we'd eventually like to automate the process of classifying images submitted by citizens who live
nearby. Perhaps we have a colleague who would like the image data processed through edge filtering, so that their special
algorithm can just work on the edge data for classification.

So, we will try to edge filter our data using some functionality from the SciPy signal processing submodule, `scipy.signal`

In [ ]:
# inspired by SciPy signal processing tutorial content: https://docs.scipy.org/doc/scipy/reference/tutorial/signal.html
import scipy.misc
import scipy.signal

# here is the image we are working with:
image = scipy.misc.face(gray=True).astype(np.float32)

fig_image = plt.figure()
ax_image = fig_image.add_subplot(111)
ax_image.imshow(image, cmap='gray')

In [ ]:
# determine the B-spline interpolation coefficients to be used for edge filtering
coefs = scipy.signal.cspline2d(image,
                               8.0) # lambda specifies the amount of "smoothing"
coefs.shape, image.shape

In [ ]:
# define the so-called "separation filter"
derfilt = np.array([1.0, -2, 1.0], dtype=np.float32)

# we now effectively calculate a second derivative to get the important / transition "edges" from the original image
deriv = (scipy.signal.sepfir2d(coefs, derfilt, [1]) + scipy.signal.sepfir2d(coefs, [1], derfilt))
fig_image = plt.figure()
ax_image = fig_image.add_subplot(111)
ax_image.imshow(deriv, cmap='gray')

Now we can send the `deriv` array to our colleague for processing (perhaps by pickling it, or using `np.savetxt` -- which may be more portable)